# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro, pearsonr, spearmanr
import os
import statistics
import seaborn as sns
from scipy.stats import norm
import plotly.express as px
from IPython.display import display_html
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import  search
%store -r PVALUE_VAR


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 

Later on is concatenated and generates the following result:

In [ ]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF.csv')
corr_df

# Analysis
From now on all the data will be analysed with the Cleaned dataframes.

## Choropleth map
This block uses the IppyWidget for the indicator list and the Choroplet for the Map. 

In this section a map is generated and painted with the values of the indicator. You can salect the indicator from the list and an new map is generated, also along 2 lists of the highest and lowest countries for the selected indicator. 




In [7]:
indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)



def globalGrapgh(indicator):
    ind = indicator[0]
    N = 10
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='RdBu',
                        width=700, height=500)

    pos_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    

    pos_styler = pos_corr.style.set_table_attributes("style='display:inline'").set_caption('Direct correlation')
    neg_styler = neg_corr.style.set_table_attributes("style='display:inline'").set_caption('Inverse correlation')

    space = "\xa0" * 10
    display_html(pos_styler._repr_html_()+ space  + neg_styler._repr_html_(), raw=True)


widgets.interactive(globalGrapgh, indicator = indicator)

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

### Choropleth for population

This map represents the population of the world. It is usefull for analysing the data if the aggregation is calculated later by the population, because the countries with a higher population have more impact in the global indicators.

In [8]:
df_gold = pd.read_csv(write_path + '/GoldDataframe.csv')
fig = px.choropleth(df_gold, locations="Country", locationmode='country names', 
                     color="Population", hover_name="Country",projection="natural earth",
                     animation_frame="Year",width=800, height=500,
                     color_continuous_scale='Reds',
                     range_color=[1000,340000000])
fig.update(layout_coloraxis_showscale=True)
fig.show()

## Country Indicators
In this codeblock a widged is implmented to generate a table with the indicators. In order to make it work just select the country from the dropdown and procede to establasih the threshold. The default value for the threshold is 0.7, which we consider to be the minimum to consider an indicator correlated to the GDP. To analyze the results:
- H0: the indicator and the GDP are uncorrelated.​
- H1: the indicator and the GDP are correlated.​

P-value: is the probability of obtaining  test results at least as extreme as the result actually observed.​
Confidence level: probability that a population parameter will fall between a set of values for a certain proportion of times. ​
Significance level:  probability of the study rejecting the null hypothesis when it is actually true.​


Confidence level is set to 1 - PVALUE_VAR% .  Significance level α = PVALUE_VAR

If p-value < α then reject  H0 and accept H1.​

The code consists in a IppyWidget with a slider and a dropdown. Once this parameters have been set it calls the method 'search' and applies a style format of the returned Dataframe.

In [9]:
def tableOut(Threshold, Country):

    df = search(Threshold, 'Country', Country)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

## Region Indicators
It's almost the same codeblock as before, but for the regions. The groups have been formed according to the Dataframe of World Data Bank.


In [10]:
def tableOut2(Threshold, Region):

    df = search(Threshold, 'Region', Region)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Region = set(df['Region'].to_list()),
    Threshold = (0, 1, 0.05))
def g(Region = 'East Asia and Pacific', Threshold = 0.7):
    return tableOut2(Threshold, Region)

    

interactive(children=(Dropdown(description='Region', index=6, options=('South Asia', 'Latin America and Caribb…

## Global Indicators
This section reperesents the final analysis of the indicators. It shows in the indicators table with the highest correlations. With this results we can give an answer to the hypothesis of the project and establish the indicators with a high GDP relation.

In [11]:
def tableOut3(Threshold):

    df = search(Threshold, 'Global')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOut3(Threshold)


interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…

## Median Global Indicators
Reading the corr_df generated at the beggining of this notebook it calculates the median of each column. This approximation is very simple and might give an idea if the aggregation method is correct. Obviously the result won't be the same but the tendency has to be similar.

Having this in mind for each column in the corr_df import the name of the indicator and compute the median. Save this new dataframe in result_df.

In [12]:
result_df = pd.DataFrame()
for column in corr_df.columns:
    aux = pd.DataFrame({'Indicator': [column],
                        'GDP Pearson Corr': [corr_df[column].median()],
                        'GDP Spearman Corr': [corr_df_spear[column].median()]})
    result_df = pd.concat([result_df, aux], ignore_index=False, axis = 0)
    result_df = result_df.sort_values(by=["GDP Pearson Corr"], ascending = False)
    

result_df.set_index(['Indicator'], inplace=True)

Using a IppyWidget display the previously generated result_df. Lowering the Threshold more than 0.7 will show indicators that don't have a real correlation with the GDP, procede with caution extracting conclusions of them.

In [13]:
def tableOutMedian(Threshold):
    df = pd.concat([result_df.loc[(result_df['GDP Pearson Corr'] >= Threshold) & (result_df['GDP Spearman Corr'] >= Threshold)], result_df.loc[(result_df['GDP Pearson Corr'] <= -Threshold) & (result_df['GDP Pearson Corr'] <= -Threshold)]], axis = 0)

    if df.empty:
        return print("No indicators have been found.")

    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\

    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOutMedian(Threshold)

interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…

0.05